In [1]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [27]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, engine

from config import local_mysql_password, local_mysql_user

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)


In [25]:
class DatabaseInterface:
    def __init__(self,
                 db_name,
                 user,
                 password,
                 host='localhost',
                 port=3306,
                 driver='mysql+pymysql'):
        
        self.con = create_engine(f'{driver}://{user}:{password}@{host}')
        self.con.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        self.db_engine = create_engine(f"{driver}://{user}:{password}@{host}:{port}/{db_name}")

    def insert_data(self, df: pd.DataFrame, table_name: str):
        df.to_sql(table_name, self.db_engine, if_exists='replace', index=False)

    def insert_data_with_types(self, df: pd.DataFrame, table_name: str, dtype: dict):
        df.to_sql(table_name, self.db_engine, if_exists='replace', index=False, dtype=dtype)
    
    def set_primary_key(self, id_column: str, table_name: str):
        self.con.execute(f'ALTER TABLE `{table_name}` ADD PRIMARY KEY (`{id_column}`) int(4) NOT NULL auto_increment FIRST;')

    def close_connection(self):
        self.db_engine.dispose()

In [2]:
living_planet_index = pd.read_csv("./data/LivingPlanetIndex_2020/Public_data_set/LPR2020data_public.csv", encoding='latin-1')
# living_planet_index.head(5)
# living_planet_index.info()

In [12]:
living_planet_index.Binomial = living_planet_index.Binomial.apply(lambda x: x.replace('_', ' '))
living_planet_index.shape

(15348, 98)

In [26]:
dbi = DatabaseInterface(db_name='climate_change_effects', 
                        user=local_mysql_user, 
                        password=local_mysql_password)
    
print("Inserting LPI data")
dbi.insert_data(living_planet_index, 'living_planet_index')
print("Closing connection")
dbi.close_connection()
print("Done")

Inserting LPI data
Closing connection
Done


In [4]:
file_name = 'simple_summary'
data = pd.read_csv(f"./data/IUCN_red_list/redlist_species_data/{file_name}.csv", encoding='latin-1')
data.shape

(7434, 19)

In [5]:
data.head(5)

,assessmentId,internalTaxonId,scientificName,kingdomName,phylumName,orderName,className,familyName,genusName,speciesName,infraType,infraName,infraAuthority,authority,redlistCategory,redlistCriteria,criteriaVersion,populationTrend,scopes
0,495907,10041,Heosemys annandalii,ANIMALIA,CHORDATA,TESTUDINES,REPTILIA,GEOEMYDIDAE,Heosemys,annandalii,NaN,NaN,NaN,"(Boulenger in Annandale &amp; Robinson, 1903)",Critically Endangered,A2cd+4cd,3.1,Decreasing,Global
1,500780,11150,Lagocephalus gloveri,ANIMALIA,CHORDATA,TETRAODONTIFORMES,ACTINOPTERYGII,TETRAODONTIDAE,Lagocephalus,gloveri,NaN,NaN,NaN,"Abe &amp; Tabeta, 1983",Data Deficient,NaN,3.1,Decreasing,Global
2,501662,11242,Lampsilis rafinesqueana,ANIMALIA,MOLLUSCA,UNIONOIDA,BIVALVIA,UNIONIDAE,Lampsilis,rafinesqueana,NaN,NaN,NaN,"Frierson, 1927",Endangered,A2ac,3.1,Decreasing,Global
3,502085,11250,Hamiota altilis,ANIMALIA,MOLLUSCA,UNIONOIDA,BIVALVIA,UNIONIDAE,Hamiota,altilis,NaN,NaN,NaN,"(Conrad, 1834)",Endangered,"B1ab(i,ii,iii,iv,v)+2ab(i,ii,iii,iv,v)",3.1,Decreasing,Global
4,505402,12124,Lissemys scutata,ANIMALIA,CHORDATA,TESTUDINES,REPTILIA,TRIONYCHIDAE,Lissemys,scutata,NaN,NaN,NaN,"(Peters, 1868)",Least Concern,NaN,3.1,Stable,Global


In [57]:
data.groupby('redlistCategory').redlistCategory.count()

redlistCategory
Critically Endangered     844
Data Deficient            683
Endangered               1256
Extinct                    30
Extinct in the Wild         2
Least Concern            2420
Near Threatened           888
Vulnerable               1311
Name: redlistCategory, dtype: int64

In [6]:
file_name = 'assessments'
data = pd.read_csv(f"./data/IUCN_red_list/redlist_species_data/{file_name}.csv", encoding='latin-1')
data.head(5)

,assessmentId,internalTaxonId,scientificName,redlistCategory,redlistCriteria,yearPublished,assessmentDate,criteriaVersion,language,rationale,habitat,threats,population,populationTrend,range,useTrade,systems,conservationActions,realm,yearLastSeen,possiblyExtinct,possiblyExtinctInTheWild,scopes
0,495907,10041,Heosemys annandalii,Critically Endangered,A2cd+4cd,2021,2018-03-13 00:00:00 UTC,3.1,English,<p><em>Heosemys annandalii</em> is considered ...,<p><em>Heosemys annandalii</em> inhabits lowla...,<p>The main threats to this species are harves...,"<p>According to Tana <em>et al</em>. (2000), t...",Decreasing,<p>The range of <em>Heosemys annandalii</em> i...,The species is collected for local consumption...,Terrestrial|Freshwater (=Inland waters),<p><em>Heosemys annandalii </em>is included in...,Indomalayan,NaN,False,False,Global
1,500780,11150,Lagocephalus gloveri,Data Deficient,NaN,2014,2011-06-08 00:00:00 UTC,3.1,English,"<em>Lagocephalus gloveri</em><span style=""font...","<span style=""font-style: italic;"">Lagocephalus...",<strong>Pufferfish fishery<br/></strong><span ...,"<span style=""font-style: italic;"">Lagacephalus...",Decreasing,"<span style=""font-style: italic;"">Lagocephalus...","<span style=""font-style: italic;"">Lagocephalus...",Marine,<p><strong>Fisheries<br/></strong><em>Lagoceph...,NaN,NaN,False,False,Global
2,501662,11242,Lampsilis rafinesqueana,Endangered,A2ac,2012,2012-06-07 00:00:00 UTC,3.1,English,<em>Lampsilis rafinesqueana</em> has been asse...,The species is found in a variety of habitats ...,"The present or threatened destruction, modific...","Currently, approximately 70 occurrences are kn...",Decreasing,This species is endemic to the Neosho and Verd...,"It was once harvested commercially; however, c...",Freshwater (=Inland waters),This species was listed as a federal candidate...,Nearctic,NaN,False,False,Global
3,502085,11250,Hamiota altilis,Endangered,"B1ab(i,ii,iii,iv,v)+2ab(i,ii,iii,iv,v)",2012,2012-03-05 00:00:00 UTC,3.1,English,<em>Hamiota altillis </em>has been assessed as...,"Historically, this species was found in large ...",Previously the species was overutilised for co...,"Historically, this species was widespread thro...",Decreasing,The type locality for this species is the Alab...,Previously the species was overutilised for co...,Freshwater (=Inland waters),This species was listed as threatened in 1993 ...,Nearctic,NaN,False,False,Global
4,505402,12124,Lissemys scutata,Least Concern,NaN,2021,2018-03-14 00:00:00 UTC,3.1,English,<em>Lissemys scutata</em> is listed as Least C...,<em>Lissemys scutata</em> is typically associa...,<p><em>Lissemys scutata</em> is traded in subs...,<em>Lissemys scutata</em> is still a common sp...,Stable,<em>Lissemys scutata</em> is endemic to Myanma...,<em>Lissemys scutata</em> is traded in substan...,Terrestrial|Freshwater (=Inland waters),<p><em>Lissemys scutata</em> is included in CI...,Indomalayan,NaN,False,False,Global


In [7]:
data[(data.redlistCategory=='Extinct') | (data.redlistCategory=='Extinct in the Wild')].head(5)

,assessmentId,internalTaxonId,scientificName,redlistCategory,redlistCriteria,yearPublished,assessmentDate,criteriaVersion,language,rationale,habitat,threats,population,populationTrend,range,useTrade,systems,conservationActions,realm,yearLastSeen,possiblyExtinct,possiblyExtinctInTheWild,scopes
406,1998432,191696,Allotoca goslinei,Extinct in the Wild,NaN,2019,2018-03-07 00:00:00 UTC,3.1,English,"Given the fact, that <em>Allotoca goslinei&#16...",Here is the habitat description from Smith and...,<em>Allotoca goslinei </em>was moderately comm...,Deduced from inconclusive surveys conducted by...,NaN,The Banded Allotoca was endemic to the Mexican...,The Banded Allotoca is not used for consumptio...,Freshwater (=Inland waters),The Banded Allotoca is included in the interna...,Neotropical,2004,False,False,Global
617,2594210,199495,Margatteoidea amoena,Extinct,NaN,2012,2007-10-25 00:00:00 UTC,3.1,English,This species has not been recorded since 1905 ...,The island where the species has been recorded...,The entire range of the species is at sea leve...,There is no information available on populatio...,Unknown,"The species is endemic to Seychelles, where it...",This species is not utilized.,Terrestrial,No conservation actions are in place for this ...,Afrotropical,1905,False,False,Global
688,2706821,201484,Orthomorpha crinita,Extinct,NaN,2014,2012-11-01 00:00:00 UTC,3.1,English,This species is known from a single specimen f...,No details are known on the habitats and ecolo...,It is likely that this species was threatened ...,This species has not been located since 1894 d...,NaN,This species was endemic to the Seychelles isl...,NaN,Terrestrial,No conservation actions were taken for this sp...,Afrotropical,1894,False,False,Global
1870,8286338,184524,Pseudamnicola desertorum,Extinct,NaN,2010,2007-10-19 00:00:00 UTC,3.1,English,The generic status of this species is uncertai...,This is a crenobiont of warm springs.,Pollution and disappearance of springs threate...,No information available.,NaN,It is recorded from the thermal springs at Dje...,NaN,Freshwater (=Inland waters),No information available.,Palearctic,19th century,False,False,"Global, Northern Africa & Pan-Africa"
2047,10304648,40091,Pachnodus velutinus,Extinct,NaN,2009,2006-01-11 00:00:00 UTC,3.1,English,Range contraction in this species has been rec...,"This snail once inhabited high forests, requi...","Being vulnerable to dessication, this species ...",NaN,NaN,"Endemic to the Seychelles, this extinct snail ...",NaN,Terrestrial,This snail was historically present in the Mor...,Afrotropical,1994,False,False,Global


In [10]:
extinct_animals.shape

(32, 23)

In [8]:
extinct_animals = data[(data.redlistCategory=='Extinct') | (data.redlistCategory=='Extinct in the Wild')]
extinct_animals.yearLastSeen.unique()

array(['2004', '1905', '1894', '19th century', '1994', '1992', nan,
       '1989', '1988', '1950s', '1975', '1799-1800', '1986', '1990',
       '1974', '1464â\x80\x931636', '1927', '1945', '1944', '1987',
       '1860', '1919', '1867', '1837', '1892', '1969', '2009', '1912'],
      dtype=object)

In [14]:
pd.merge(living_planet_index, extinct_animals,left_on='Binomial',right_on='scientificName')

,ID,Binomial,Reference,Citation,Class,Order,Family,Genus,Species,Subspecies,Common_name,Location,Country,All_countries,Region,Latitude,Longitude,Specific_location,temperate_or_tropical,System,T_realm,T_biome,FW_realm,FW_biome,M_realm,M_ocean,M_biome,Units,Method,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,assessmentId,internalTaxonId,scientificName,redlistCategory,redlistCriteria,yearPublished,assessmentDate,criteriaVersion,language,rationale,habitat,threats,population,populationTrend,range,useTrade,systems,conservationActions,realm,yearLastSeen,possiblyExtinct,possiblyExtinctInTheWild,scopes
0,6475,Atelopus longirostris,"{La Marca, 2005 #1718}","La Marca, E., K. R. Lips, et al. (2005). Cata...",Amphibia,Anura,Bufonidae,Atelopus,longirostris,NaN,Longnose stubfoot toad,Rio Faisanes,Ecuador,Ecuador,Latin America and Caribbean,-0.25056,-78.80222,No,Tropical,Freshwater,NaN,NaN,Neotropical,Tropical and subtropical coastal rivers,NaN,NaN,NaN,individuals/person/hour,surveys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11158637,54522,Atelopus longirostris,Extinct,NaN,2004,2004-04-30 00:00:00 UTC,3.1,English,Listed as Extinct because it has not been reco...,This was a terrestrial species of lowland and ...,"The decline in Ecuador is unexplained, and is ...","The species appears to be extinct, with no rec...",NaN,This species was recorded from the Provinces o...,NaN,Terrestrial|Freshwater (=Inland waters),The range of this species overlapped with the ...,Neotropical,1989,False,False,Global
1,6478,Incilius periglenes,"{Crump, 1992 #1719}","Crump, M. L., F. R. Hensley, et al. (1992). A...",Amphibia,Anura,Bufonidae,Incilius,periglenes,NaN,Golden toad,Monteverde Cloud Forest Reserve.,Costa Rica,Costa Rica,Latin America and Caribbean,10.28333,-84.80000,No,Tropical,Freshwater,NaN,NaN,Neotropical,Tropical and subtropical coastal rivers,NaN,NaN,NaN,Total number of individuals,Visual searches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,10.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54357699,3172,Incilius periglenes,Extinct,NaN,2020,2019-09-11 00:00:00 UTC,3.1,English,Listed as Extinct because it has not been reco...,"This species inhabited cloud and elfin forest,...","Its restricted range, climate change, chytridi...","Formerly a common species, no specimen has bee...",NaN,This species was known only from the Reserva B...,NaN,Terrestrial|Freshwater (=Inland waters),The entire range of this species was protected...,Neotropical,1989,False,False,Global


In [15]:
pd.merge(living_planet_index,data,left_on='Binomial',right_on='scientificName').shape

(3474, 121)

In [70]:
pd.merge(living_planet_index,data,left_on='Binomial',right_on='scientificName').groupby('redlistCategory').redlistCategory.count()

redlistCategory
Critically Endangered     181
Data Deficient             58
Endangered                399
Extinct                     2
Least Concern            1844
Near Threatened           398
Vulnerable                592
Name: redlistCategory, dtype: int64

In [17]:

file_name = 'Production_Crops_Livestock_E_All_Data'
data = pd.read_csv(f"./data/crop_data/faostat_Production_Crops_Livestock_E_All_Data/{file_name}.csv", encoding='latin-1')
data.head(5)

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,Y1962F,Y1963,Y1963F,Y1964,Y1964F,Y1965,Y1965F,Y1966,Y1966F,Y1967,Y1967F,Y1968,Y1968F,Y1969,Y1969F,Y1970,Y1970F,Y1971,Y1971F,Y1972,Y1972F,Y1973,Y1973F,Y1974,Y1974F,Y1975,Y1975F,Y1976,Y1976F,Y1977,Y1977F,Y1978,Y1978F,Y1979,Y1979F,Y1980,Y1980F,Y1981,Y1981F,Y1982,Y1982F,Y1983,Y1983F,Y1984,Y1984F,Y1985,Y1985F,Y1986,Y1986F,Y1987,Y1987F,Y1988,Y1988F,Y1989,Y1989F,Y1990,Y1990F,Y1991,Y1991F,Y1992,Y1992F,Y1993,Y1993F,Y1994,Y1994F,Y1995,Y1995F,Y1996,Y1996F,Y1997,Y1997F,Y1998,Y1998F,Y1999,Y1999F,Y2000,Y2000F,Y2001,Y2001F,Y2002,Y2002F,Y2003,Y2003F,Y2004,Y2004F,Y2005,Y2005F,Y2006,Y2006F,Y2007,Y2007F,Y2008,Y2008F,Y2009,Y2009F,Y2010,Y2010F,Y2011,Y2011F,Y2012,Y2012F,Y2013,Y2013F,Y2014,Y2014F,Y2015,Y2015F,Y2016,Y2016F,Y2017,Y2017F,Y2018,Y2018F,Y2019,Y2019F,Y2020,Y2020F
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,ha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,F,5900.0,F,6000.0,F,6000.0,F,6000.0,F,5800.0,F,5800.0,F,5800.0,F,5700.0,F,5700.0,F,5600.0,F,5500.0,F,5500.0,F,5400.0,F,5400.0,F,6037.0,Im,5500.0,NaN,5500.0,NaN,5500.0,NaN,5500.0,NaN,5500.0,NaN,5500.0,NaN,5500.0,NaN,5500.0,NaN,5500.0,NaN,7000.0,F,9000.0,F,5500.0,NaN,5700.0,NaN,12000.0,*,11768.0,Im,12000.0,NaN,12000.0,NaN,12000.0,NaN,11029.0,NaN,11210.0,NaN,13469.0,NaN,13490.0,NaN,14114.0,NaN,13703.0,NaN,14676.0,NaN,19481.0,NaN,19793.0,NaN,20053.0,NaN,29203.0,NaN,22134.0,NaN
1,2,Afghanistan,221,"Almonds, with shell",5419,Yield,hg/ha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16610.0,Fc,15000.0,Fc,20000.0,Fc,17500.0,Fc,17069.0,Fc,13793.0,Fc,18966.0,Fc,17018.0,Fc,18421.0,Fc,16071.0,Fc,18182.0,Fc,16364.0,Fc,16667.0,Fc,16296.0,Fc,15736.0,Fc,16364.0,Fc,18000.0,Fc,16364.0,Fc,16364.0,Fc,16364.0,Fc,16364.0,Fc,16364.0,Fc,16364.0,Fc,20000.0,Fc,17143.0,Fc,16667.0,Fc,21407.0,Fc,24561.0,Fc,12250.0,Fc,13282.0,Fc,16667.0,Fc,26234.0,Fc,35000.0,Fc,39154.0,Fc,49955.0,Fc,45000.0,Fc,45960.0,Fc,29910.0,Fc,19996.0,Fc,16521.0,Fc,16859.0,Fc,13788.0,Fc,17161.0,Fc,13083.0,Fc,17759.0,Fc
2,2,Afghanistan,221,"Almonds, with shell",5510,Production,tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,F,9800.0,F,9000.0,F,12000.0,F,10500.0,F,9900.0,F,8000.0,F,11000.0,F,9700.0,F,10500.0,F,9000.0,F,10000.0,F,9000.0,F,9000.0,F,8800.0,F,9500.0,F,9000.0,NaN,9900.0,NaN,9000.0,NaN,9000.0,NaN,9000.0,NaN,9000.0,NaN,9000.0,NaN,9000.0,NaN,11000.0,F,12000.0,F,15000.0,F,11774.0,Im,14000.0,*,14700.0,*,15630.0,*,20000.0,NaN,31481.0,NaN,42000.0,NaN,43183.0,*,56000.0,NaN,60611.0,NaN,62000.0,NaN,42215.0,NaN,27400.0,NaN,24246.0,NaN,32843.0,NaN,27291.0,NaN,34413.0,NaN,38205.0,NaN,39307.0,NaN
3,2,Afghanistan,711,"Anise, badian, fennel, coriander",5312,Area harvested,ha,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,NaN,M,700.0,F,700.0,F,300.0,F,1100.0,F,1300.0,F,1882.0,Im,700.0,F,2270.0,Im,2871.0,Im,3000.0,F,7000.0,F,4000.0,F,12000.0,F,11311.0,Im,4000.0,F,1600.0,F,3300.0,F,6800.0,F,15000.0,F,16000.0,F,17432.0,Im,28000.0,F,15000.0,F,17748.0,Im,17000.0,F,19500.0,Im,18500.0,F,18500.0,F,30000.0,F,25000.0,F,24500.0,Im,26500.0,Im,25333.0,Im,25444.0,Im,25759.0,Im
4,2,Afghanistan,711,"Anise, badian, fennel, coriander",5419,Yield,hg/ha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7143.0,Fc,7143.0,Fc,6667.0,Fc,7273.0,Fc,7692.0,Fc,7072.0,Fc,7143.0,Fc,6855.0,Fc,6761.0,Fc,6667.0,Fc,6000.0,Fc,6250.0,Fc,5917.0,Fc,6189.0,Fc,6250.0,Fc,6250.0,Fc,6061.0,Fc,6029.0,Fc,6000.0,Fc,6250.0,Fc,6222.0,Fc,6071.0,Fc,6000.0,Fc,6202.0,Fc,6000.0,Fc,6414.0,Fc,6757.0,Fc,6757.0,Fc,7167.0,Fc,7200.0,Fc,7075.0,Fc,7149.0,Fc,7142.0,F

In [18]:
data.shape

(77910, 127)

In [34]:
data.Area.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belgium-Luxembourg', 'Belize',
       'Benin', 'Bhutan', 'Bolivia (Plurinational State of)',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China',
       'China, Hong Kong SAR', 'China, Macao SAR', 'China, mainland',
       'China, Taiwan Province of', 'Colombia', 'Comoros', 'Congo',
       'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba',
       'Cyprus', 'Czechia', 'Czechoslovakia',
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
      

In [29]:
data[data.Y1961F=='A']

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,Y1962F,Y1963,Y1963F,Y1964,Y1964F,Y1965,Y1965F,Y1966,Y1966F,Y1967,Y1967F,Y1968,Y1968F,Y1969,Y1969F,Y1970,Y1970F,Y1971,Y1971F,Y1972,Y1972F,Y1973,Y1973F,Y1974,Y1974F,Y1975,Y1975F,Y1976,Y1976F,Y1977,Y1977F,Y1978,Y1978F,Y1979,Y1979F,Y1980,Y1980F,Y1981,Y1981F,Y1982,Y1982F,Y1983,Y1983F,Y1984,Y1984F,Y1985,Y1985F,Y1986,Y1986F,Y1987,Y1987F,Y1988,Y1988F,Y1989,Y1989F,Y1990,Y1990F,Y1991,Y1991F,Y1992,Y1992F,Y1993,Y1993F,Y1994,Y1994F,Y1995,Y1995F,Y1996,Y1996F,Y1997,Y1997F,Y1998,Y1998F,Y1999,Y1999F,Y2000,Y2000F,Y2001,Y2001F,Y2002,Y2002F,Y2003,Y2003F,Y2004,Y2004F,Y2005,Y2005F,Y2006,Y2006F,Y2007,Y2007F,Y2008,Y2008F,Y2009,Y2009F,Y2010,Y2010F,Y2011,Y2011F,Y2012,Y2012F,Y2013,Y2013F,Y2014,Y2014F,Y2015,Y2015F,Y2016,Y2016F,Y2017,Y2017F,Y2018,Y2018F,Y2019,Y2019F,Y2020,Y2020F
194,2,Afghanistan,1806,Beef and Buffalo Meat,5510,Production,tonnes,43000.0,A,45800.0,A,47250.0,A,48000.0,A,48700.0,A,68000.0,A,65000.0,A,71000.0,A,75000.0,A,62000.0,A,62000.0,A,61000.0,A,63000.0,A,62000.0,A,61000.0,A,62300.0,A,66000.0,A,67000.0,A,71000.0,A,67000.0,A,68000.0,A,69000.0,A,68000.0,A,68000.0,A,68000.0,A,68000.0,A,70000.0,A,71000.0,A,71000.0,A,86400.0,A,86400.0,A,86400.0,A,97200.0,A,113400.0,A,130000.0,A,143000.0,A,156000.0,A,171000.0,A,149000.0,A,126000.0,A,108000.0,A,150000.0,A,137000.0,A,149300.0,A,141100.0,A,117300.0,A,137000.0,A,133000.0,A,135000.0,A,131000.0,A,138000.0,A,139000.0,A,134000.0,A,120545.0,A,111438.0,A,107533.0,A,114994.0,A,115788.0,A,98204.0,A,116008.0,A
195,2,Afghanistan,1806,Beef and Buffalo Meat,5320,Producing Animals/Slaughtered,Head,360000.0,A,384000.0,A,396000.0,A,402000.0,A,408000.0,A,569600.0,A,544000.0,A,595300.0,A,628300.0,A,653000.0,A,653000.0,A,643200.0,A,563200.0,A,520300.0,A,511500.0,A,523500.0,A,541000.0,A,557000.0,A,590000.0,A,560000.0,A,560000.0,A,560000.0,A,550000.0,A,545000.0,A,540000.0,A,535000.0,A,550000.0,A,560000.0,A,560000.0,A,480000.0,A,480000.0,A,480000.0,A,540000.0,A,630000.0,A,722000.0,A,794000.0,A,866000.0,A,950000.0,A,828000.0,A,700000.0,A,600000.0,A,833000.0,A,761000.0,A,829000.0,A,784000.0,A,651700.0,A,761100.0,A,739000.0,A,750000.0,A,727800.0,A,766700.0,A,772000.0,A,744500.0,A,669717.0,A,619065.0,A,671906.0,A,638865.0,A,643279.0,A,651350.0,A,644498.0,A
196,2,Afghanistan,1811,Butter and Ghee,5510,Production,tonnes,11104.0,A,11124.0,A,12415.0,A,12725.0,A,13880.0,A,14421.0,A,15725.0,A,16385.0,A,16385.0,A,14388.0,A,13110.0,A,13540.0,A,15014.0,A,15853.0,A,16405.0,A,16860.0,A,15276.0,A,15470.0,A,15841.0,A,16203.0,A,16723.0,A,16608.0,A,17535.0,A,16725.0,A,13825.0,A,8813.0,A,10125.0,A,10575.0,A,11025.0,A,15982.0,A,16793.0,A,17356.0,A,19688.0,A,23395.0,A,26253.0,A,29925.0,A,32730.0,A,34068.0,A,38940.0,A,31465.0,A,20731.0,A,34992.0,A,32707.0,A,33810.0,A,32969.0,A,31078.0,A,33611.0,A,32702.0,A,33149.0,A,33001.0,A,33248.0,A,35069.0,A,35723.0,A,47281.0,A,42365.0,A,42496.0,A,41102.0,A,41778.0,A,44084.0,A,NaN,NaN
197,2,Afghanistan,1746,Cattle and Buffaloes,5111,Stocks,Head,2900000.0,A,3200000.0,A,3300000.0,A,3350000.0,A,3400000.0,A,3600000.0,A,3600000.0,A,3633000.0,A,3600000.0,A,3700000.0,A,3680000.0,A,3400000.0,A,3232000.0,A,3433000.0,A,3639000.0,A,3700000.0,A,3650000.0,A,3730000.0,A,3710000.0,A,3710000.0,A,3750000.0,A,3740000.0,A,3500000.0,A,2700000.0,A,2200000.0,A,1532000.0,A,1580000.0,A,1600000.0,A,1600000.0,A,1600000.0,A,1600000.0,A,1700000.0,A,1800000.0,A,1900000.0,A,2094560.0,A,2641400.0,A,2895000.0,A,3008330.0,A,3477549.0,A,2900000.0,A,2249000.0,A,3715000.0,A,3829000.0,A,3494000.0,A,3723000.0,A,4110000.0,A,4357000.0,A,4745000.0,A,4721000.0,A,5673000.0,A,5524000.0,A,5244000.0,A,5235000.0,A,5349000.0,A,5261000.0,A,5234400.0,A,4977000.0,A,5157467.0,A,5122956.0,A,5085807.0,A
198,2,Afghanistan,1717,"Cereals, Total",5312,Area harvested,ha,3313500.0,A,3425500.0,A,3426500.0,A,3446500.0,A,3444500.0,A,3446500.0,A,3124500.0,A,3041500.0,A,3081500.0,A,3178000.0,A,3400000.0,A,3923100.0,A,3337000.0,A,3342000.0,A,3404000.0,A,3394000.0,A,3388000.0,A,3392000.0,A,31835

In [ ]:
flag_descriptions = {
    np.nan: 'Official data', 
    'Fc': 'Calculated data', 
    'Im': 'FAO data based on imputation methodology', 
    'F': 'FAO estimate', 
    '*': 'Unofficial figure', 
    'A': 'Aggregate, may include official, semi-official, estimated or calculated data', 
    'M': 'Data not available'
}